In [53]:
import pandas as pd
import numpy as np

In [54]:
df = pd.read_csv('rev2.csv')

In [124]:
len(df.index)

12482

In [56]:
df.tail()

,product_id,review,rating
12478,46679049,ürün çok hızlı bir şekilde ulaştı firmaya tşk ...,5
12479,46679049,öncelikle çok teşekkürler ☺️ ambalaj çok güzel...,5
12480,46679049,sağlam hasarsız şekilde geldi sorunsuz çalışıy...,5
12481,46679049,Ürün çok hızlı Bi şekilde elime ulaştı teşekkü...,5
12482,46679049,Paketleme ve gönderi çok hızlı ve güzel satıcı...,5


#### Transform to lower case

In [57]:
df['lower'] = df['review'].str.lower()

#### Remove punctuation

In [59]:
df['wo_punct'] = df['lower'].str.replace('[^\w\s]','')

**Remove the whitespaces at the end of each review**

In [61]:
df['clean_reviews'] = df['wo_punct'].str.strip()

**Remove digits**

In [62]:
df['reviews'] = df['clean_reviews'].str.replace('\d+', '')

**Drop other columns. Reviews are the column with the clean data**

In [63]:
df.drop(columns = ["lower", "wo_punct", "clean_reviews"], inplace = True)

In [64]:
df.head()

,product_id,review,rating,reviews
0,31722470,5 gündür 3 er kapsül kullanıyorum. Ne tokluk v...,1,gündür er kapsül kullanıyorum ne tokluk veri...
1,31722470,şiddetli başağrısı yaptı ağrı hassasiyeti olan...,1,şiddetli başağrısı yaptı ağrı hassasiyeti olan...
2,31722470,isi yaramıyo (((,1,isi yaramıyo
3,31722470,Hiç bir etkisi olmadı,1,hiç bir etkisi olmadı
4,31722470,maalesef bana iyi gelmedi,1,maalesef bana iyi gelmedi


In [66]:
df.reviews.isnull().any()

False

In [67]:
df.to_csv("clean_tyol_data.csv", index=False)

### Import required modules

In [126]:
import pandas as pd
import numpy as np
import nltk


Create a stopwords list for better accuracy

In [127]:
from nltk.corpus import stopwords

In [128]:
stop_word_list = nltk.corpus.stopwords.words('turkish')

### Load the clean data

In [129]:
df = pd.read_csv('clean_tyol_data.csv')

In [130]:
df.dtypes

product_id     int64
review        object
rating         int64
reviews       object
dtype: object

Drop the row with only emojis

In [131]:
df.loc[df.reviews.isnull()]

,product_id,review,rating,reviews
3344,40275334,🥳🥳🥳🥳🥳🥳🥳🥳🥳🥳,5,NaN


In [132]:
df.dropna(inplace = True)

In [133]:
df.loc[df.reviews.isnull()]

,product_id,review,rating,reviews


In [134]:
df.head()

,product_id,review,rating,reviews
0,31722470,5 gündür 3 er kapsül kullanıyorum. Ne tokluk v...,1,gündür er kapsül kullanıyorum ne tokluk veri...
1,31722470,şiddetli başağrısı yaptı ağrı hassasiyeti olan...,1,şiddetli başağrısı yaptı ağrı hassasiyeti olan...
2,31722470,isi yaramıyo (((,1,isi yaramıyo
3,31722470,Hiç bir etkisi olmadı,1,hiç bir etkisi olmadı
4,31722470,maalesef bana iyi gelmedi,1,maalesef bana iyi gelmedi


In [135]:
df.rating.value_counts()

5    3389
4    2670
1    2343
3    2273
2    1807
Name: rating, dtype: int64

In [140]:
reviews = []
for i in df.index:
    reviews.append((df['reviews'][i], df['rating'][i]))

In [95]:
reviews = []
for i in df.index:
    reviews.append(Review(df['reviews'][i], df['rating'][i]))

In [96]:
reviews[127].text

' gün de kargoya verildi  ayda teslim aldım kalitesi fena değil kargo hızı rezalet'

In [97]:
reviews[127].rating

2

### Divide the data for training and test

In [139]:
from sklearn.model_selection import train_test_split

In [141]:
training, test = train_test_split(reviews, test_size = 0.33, random_state = 42)

In [143]:
train_x = []
train_y = []
for i in training:
    train_x.append(i[0])
    train_y.append(i[1])
    

In [145]:
test_x = []
test_y = []
for i in test:
    test_x.append(i[0])
    test_y.append(i[1])

### Bag of Words Vectorization

In [146]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words=stop_word_list)



In [147]:
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)

### Adopt svm model for classification

In [148]:
from sklearn import svm
clf_svm = svm.SVC(kernel = "linear")

In [149]:
clf_svm.fit(train_x_vectors, train_y)

SVC(kernel='linear')

In [157]:
test_x[300]

'silerken çok leke kalıyor kalitesinden çok memnun kalmadım'

In [158]:
clf_svm.predict(test_x_vectors[300])

array([2])

### Not the best needs lots of tweak, maybe another classification method and more data

In [159]:
clf_svm.score(test_x_vectors, test_y)

0.4286407766990291

In [166]:
test_set = ["Beğendim ancak tamamen sorunsuz değil. Kurulumu epey uğraştırdı. Yine de işe yarar", "Yenecek gibi değil çöpe attık", "idare eder ama tekrar sipariş vereceğimi sanmam", "oldukça beğendim, kızıma tam oldu"]

new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array([3, 2, 3, 5])